In [1]:
import re
import numpy as np
import time
import os.path
import sys
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from io import StringIO
from Bio import SeqIO
import pysam
from pandas.plotting import scatter_matrix
import warnings
import json
import numpy as np
import subprocess
warnings.filterwarnings('ignore')
sys.executable.split('/')[-3]

'blastasm'

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
def get_fasta_ids(sra_file):
    idlist=[]
    for record in SeqIO.parse(sra_file, "fasta"):
        idlist.append(record.id)
    return idlist

In [4]:
def read_bam(in_path, bam_file):
    if bam_file.endswith('.bam'):
        samfile = pysam.AlignmentFile(in_path+bam_file, "rb")
    elif bam_file.endswith('.sam'):
        samfile = pysam.AlignmentFile(in_path+bam_file, "r")
    stats=samfile.get_index_statistics()
    #print(stats)
    return samfile

In [5]:
def write_df(data_type, stats, aln_presequence, aln_params, align_1, align_2):
    df=pd.DataFrame(stats).T
    df=pd.DataFrame(stats)
    df.columns=[align_1,align_2, align_2+'_not_'+align_1, align_1+'_not_'+align_2,'common_ids']
    #df2=pd.DataFrame([SRAs, [align_1]*len(SRAs), [align_2]*len(SRAs)]).T
    #df2.columns=['SRA','aln_1','aln_2']
    #df = pd.concat([df2, df1], axis=1, join='inner')
    df.insert(0, 'SRA', SRAs)
    df.to_csv(f'{BASE_PATH}/general_plots/{data_type}{aln_presequence}_{aln_params}_{align_1}_and_{align_2}_simplified.csv')

# ConcatRef


Each SRA of raw reads and SRA of de novo contigs aligned to reference 'ConcatRef'

Best alignment wins

In [6]:
def run_flagtools(bam_file, stats_file):
    cmd=f"{SAMTOOLS_PATH}samtools flagstat {bam_file} > {stats_file}"
    try:
        subprocess.check_call(cmd, shell=True)
    except subprocess.CalledProcessError as e:
        raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

In [7]:
def write_sam(input_path, input_file, chr_list, out_file, out_type):
    #w for sam, wb for bam
    if input_file.endswith('.bam'):
        sam = pysam.AlignmentFile(input_path+input_file, 'rb')
    elif input_file.endswith('.sam'):
        sam = pysam.AlignmentFile(input_path+input_file, 'r')
    else:
        raise ValueError('Unknown file type')
    if out_type=='sam':
        bin_code='w'
    else:
        bin_code='wb'
    with pysam.AlignmentFile(input_path+out_file, bin_code, header=sam.header) as outf:
        if sam is not None:
            for record in sam:
                if record.reference_name in chr_list:
                    outf.write(record)

In [8]:
def concatat_ref_split(aln_presequence, concat_ref, aln_postfix, ref_ids, split_ref_name, out_postfix, in_type='bam', out_type='bam'):
    for sra in SRAs:
        bam_path=BASE_PATH+sra+'/'+ASSEMBLY+'/'   
        bam_f=f'{sra}_{aln_presequence}_{concat_ref}_{aln_postfix}.{in_type}'
        out_sam=f'{sra}_{aln_presequence}_{concat_ref}_{out_postfix}_ConcatRef_split_{split_ref_name}.{out_type}'
        write_sam(bam_path, bam_f, ref_ids, out_sam, out_type)
        stats_file=out_sam.split('.'+out_type)[0]+'_flagstats.txt'
        run_flagtools(bam_path+out_sam, stats_file)

In [14]:
PRJ='PRJNA901878'
BASE_PATH = f'/mnt/8TB_2/Data/Assembly/{PRJ}/'

REF_BASE_PATH='/mnt/1TB_0/Data/fasta/mammal_genomes/'

REFERENCE_MJ='GCF_014570535.1_YNU_ManJav_2.0_genomic.fna'
REFERENCE_S='GCF_000003025.6_Sscrofa11.1_genomic.fna'
REFERENCE_RP='GCA_009823505.1_RhiPru_1.0_genomic.fna'
REFERENCE_H='GCF_000001405.39_GRCh38.p13_genomic.fna'
REFRENCE_V='GCF_015252025.1_Vero_WHO_p1.0_genomic.fna'

#_x_sr_secondary_no_sus_vero_mj_hu_minimap2
ASSEMBLY='minimap2'
BAM_MIDFIX='_sus_manjav_'
BAM_POSTFIX='_minimap2_sorted.sam'

SAMTOOLS_PATH=''

In [45]:
SRAs=['SRR22936419','SRR22936420','SRR22936421','SRR22936422',\
      'SRR22936541','SRR22936544','SRR22936770','SRR22936773']

In [46]:
#ALN_TYPE='reads'
#ALN_PRESEQ='x_sr_secondary_no'
#CONCAT_REF='sus_mj'
#ALN_POSTFIX='minimap2_sorted'
#OUT_POSTFIX=ALN_TYPE+'_default_sr_minimap2'

ALN_TYPE='contigs'
aln_presequence='fastp_MEGAHIT_final_min_len_300'
concat_ref='mj_sus'
aln_postfix='default_secondary_no_minimap2_sorted'
out_postfix=ALN_TYPE+'_min_len_300_default_sr_minimap2'

In [47]:
ref_ids=get_fasta_ids(REF_BASE_PATH+REFERENCE_S)
split_ref_name='sus'

In [48]:
stats_s=concatat_ref_split(aln_presequence, concat_ref, aln_postfix, ref_ids, split_ref_name, out_postfix, in_type='bam', out_type='bam')

In [49]:
ref_ids=get_fasta_ids(REF_BASE_PATH+REFERENCE_MJ)
split_ref_name='mj'
stats_mj=concatat_ref_split(aln_presequence, concat_ref, aln_postfix, ref_ids, split_ref_name, out_postfix, in_type='bam', out_type='bam')

In [15]:
cmd=f"bash /mnt/4TB_1/Data/Assembly/PRJNA901878/scripts/sam_proc_mj_reads.sh"
try:
    subprocess.check_call(cmd, shell=True)
except subprocess.CalledProcessError as e:
    raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

In [16]:
cmd=f"bash /mnt/4TB_1/Data/Assembly/PRJNA901878/scripts/sam_proc.sh"
try:
    subprocess.check_call(cmd, shell=True)
except subprocess.CalledProcessError as e:
    raise RuntimeError("command '{}' return with error (code {}): {}".format(e.cmd, e.returncode, e.output))

### bamboo rat

In [15]:
#_x_sr_secondary_no_sus_vero_mj_hu_minimap2
ASSEMBLY='minimap2'
BAM_MIDFIX='_rhipru_sus_'
BAM_POSTFIX='_minimap2_sorted.sam'
aln_presequence='fastp_MEGAHIT_final_min_len_300'
concat_ref='rhipru_sus'
aln_postfix='default_secondary_no_minimap2_sorted'
out_postfix='MEGAHIT_min_len_300_default_sr_minimap2'

In [16]:
SRAs=['SRR22936497']

In [17]:
ref_ids=get_fasta_ids(REF_BASE_PATH+REFERENCE_RP)
split_ref_name='rhipru'
stats_rp=concatat_ref_split(aln_presequence, concat_ref, aln_postfix, ref_ids, split_ref_name, out_postfix, in_type='bam', out_type='bam')

In [18]:
ref_ids=get_fasta_ids(REF_BASE_PATH+REFERENCE_S)
split_ref_name='sus'
stats_s=concatat_ref_split(aln_presequence, concat_ref, aln_postfix, ref_ids, split_ref_name, out_postfix, in_type='bam', out_type='bam')